# 네이버 뉴스 크롤링 - Graph DB 데이터 수집

네이버 뉴스에서 분야별 헤드라인 기사 수집

- 경제: https://news.naver.com/section/101
- IT/과학: https://news.naver.com/section/105

## 수집 항목 (Articles.xlsx)
- article_id: 기사 고유 id
- title: 기사 제목
- content: 기사 본문
- url: 기사 url
- published_date: 발행일
- source: 언론사
- author: 기자명
- category: 카테고리

## 1. 필요한 패키지 설치 및 임포트

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
from datetime import datetime
import re

## 2. 카테고리 설정 및 Selenium 초기화

In [2]:
categories = {
    '경제': 'https://news.naver.com/section/101',
    'IT/과학': 'https://news.naver.com/section/105'
}

NUM_ARTICLES_PER_CATEGORY = 10

In [3]:
service = Service(ChromeDriverManager().install())              # 설치돼있는 크롬을 불러온다.
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')                            
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=service, options=options)

## 3. 기사 수집 함수 정의

In [4]:
def get_article_links(driver, category_url, num_articles):
    driver.get(category_url)
    time.sleep(3)

    article_links = []

    try:
        selectors = [
            'a.sa_text_lede',
            'a.sa_text_strong',
            '.sa_text a',
            '.cluster_text_headline a',
            '.cluster_text_lede a'
        ]

        for selector in selectors:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for element in elements:
                url = element.get_attribute('href')
                if (url and 'news.naver.com' in url and '/article/' in url
                    and '/comment/' not in url  # 댓글 페이지만 제외
                    and url not in article_links):
                    article_links.append(url)
                    if len(article_links) >= num_articles:
                        break
            if len(article_links) >= num_articles:
                break
        print(f'✅ {len(article_links)}개의 기사 링크 수집 완료')
    except Exception as e:
        print(f'❎ 기사 링크 수집 실패: {e}')
    
    return article_links[:num_articles]

In [5]:
def parse_article_detail(driver, article_url, category):
    driver.get(article_url)
    time.sleep(1.5)

    article_data = {
        'article_id': '',
        'title': '',
        'content': '',
        'url': article_url,
        'published_date': '',
        'source': '',
        'author': '',
        'category': category
    }

    try:
        # 기사 ID 생성 (url에서 추출)
        match = re.search(r'article/(₩d+)/(₩d+)', article_url)
        if match:
            article_data['article_id'] = f"ART_{match.group(1)}_{match.group(2)}"
        else:
            article_data['article_id'] = f"ART_{datetime.now().strftime('%Y%m%d%H%M%S')}"
        
        # 제목
        title_selectors = [
            '#title_area span',
            '#ct .media_end_head_headline',
            '.media_end_head_headline',
            'h2#title_area',
            '.news_end_title'
        ]

        for selector in title_selectors:
            try:
                title_element = driver.find_element(By.CSS_SELECTOR, selector)
                if title_element.text.strip():
                    article_data['title'] = title_element.text.strip()
            except:
                continue

        # 본문
        content_selectors = [
            '#dic_area',
            'article#dic_area',
            '.go_trans._article_content',
            '._article_body_contents'
        ]

        for selector in content_selectors:
            try:
                content_element = driver.find_element(By.CSS_SELECTOR, selector)
                if content_element.text.strip():
                    article_data['content'] = content_element.text.strip()
            except:
                continue
        
        # 언론사
        try:
            source_element = driver.find_element(By.CSS_SELECTOR, 'a.media_end_head_top_logo img')
            article_data['source'] = source_element.get_attribute('alt')
        except:
            try:
                source_element = driver.find_element(By.CSS_SELECTOR, 'media_end_head_top_logo img')
                article_data['source'] = source_element.text.strip()
            except:
                pass
        
        # 발행일
        try:
            date_element = driver.find_element(By.CSS_SELECTOR, '' \
            'span.media_end_head_info_datestamp_time, span[data-date-time]')
            date_text = date_element.get_attribute('data-date-time') or date_element.txt
            article_data['published-date'] = date_text.strip()
        except:
            article_data['published_date'] = datetime.now().strftime('%Y-%m-%d %H:%M')
        
        # 기자명
        try:
            author_element = driver.find_element(By.CSS_SELECTOR, 'em.media_end_head_journalist_name, span.byline_s')
            article_data['author'] = author_element.text.strip()
        except:
            pass
    
    except Exception as e:
        print(f'❎ 파싱 오류: {e}')
    
    return article_data


## 4. 전체 카테고리 크롤링 실행

In [6]:
# 전체 기사 데이터를 저장할 리스트
all_articles = []

# 카테고리별 크롤링
for category_name, category_url in categories.items():
    print(f"\n{'='*60}")
    print(f"[{category_name}] 카테고리 수집 시작...")
    print(f"{'='*60}")

    # 1단계: 기사 링크 수집
    article_links = get_article_links(driver, category_url, NUM_ARTICLES_PER_CATEGORY)

    # 2단계: 각 기사 상세 정보 수집
    for idx, article_url in enumerate(article_links, 1):
        print(f"  [{idx}/{len(article_links)}] {article_url}")
        article_data = parse_article_detail(driver, article_url, category_name)

        if article_data['title']:   # 제목이 있는 경우만 추가
            all_articles.append(article_data)
            print(f"  ✅ 수집 완료:  {article_data['title'][:50]}...")
        else:
            print(f"  ❎ 수집 실패 - 제목을 찾을 수 없습니다.")
        
        time.sleep(0.5)


[경제] 카테고리 수집 시작...
✅ 10개의 기사 링크 수집 완료
  [1/10] https://n.news.naver.com/mnews/article/374/0000491272
  ✅ 수집 완료:  HD현대일렉, 작년 영업익 1조 육박…AI 인프라 투자 훈풍...
  [2/10] https://n.news.naver.com/mnews/article/009/0005633336
  ✅ 수집 완료:  “프장 열리자마자 공포”...삼전·두빌·기아까지 하한가 쏟아졌다고?...
  [3/10] https://n.news.naver.com/mnews/article/009/0005633714
  ✅ 수집 완료:  "원화약세 막자"… 정부, 외평채 17년만에 최대규모 30억弗 발행...
  [4/10] https://n.news.naver.com/mnews/article/014/0005474557
  ✅ 수집 완료:  9일부터 스마트폰으로 로또 산다… 20여년 묶인 빗장 풀려...
  [5/10] https://n.news.naver.com/mnews/article/052/0002311363
  ✅ 수집 완료:  코스피 낙폭 줄여 5,000선 회복...한때 매도 사이드카...
  [6/10] https://n.news.naver.com/mnews/article/021/0002769459
  ✅ 수집 완료:  반도체·서학개미의 힘… 작년 경상흑자 1230억달러 사상최대...
  [7/10] https://n.news.naver.com/mnews/article/277/0005717655
  ✅ 수집 완료:  당기순익 5.8조 KB, '번 만큼 돌려준다'…역대 최대 규모 주주환원...
  [8/10] https://n.news.naver.com/mnews/article/052/0002311480
  ✅ 수집 완료:  쿠팡 변수에 '새벽 배송' 빗장 풀리나?...여당 내 이견도...
  [9/10] https://n.news.naver.com/mnews/article/437/0

In [7]:
df_articles = pd.DataFrame(all_articles)

In [8]:
df_articles

,article_id,title,content,url,published_date,source,author,category,published-date
0,ART_20260206195511,"HD현대일렉, 작년 영업익 1조 육박…AI 인프라 투자 훈풍",AI 데이터센터 특수·노후 전력망 교체 효과…영업익 48.8% 급증\n[HD현대일렉...,https://n.news.naver.com/mnews/article/374/000...,,SBS Biz,조슬기 기자,경제,2026-02-06 15:51:08
1,ART_20260206195513,“프장 열리자마자 공포”...삼전·두빌·기아까지 하한가 쏟아졌다고?,프리마켓 개장 초반 하한가 속출\n접속매매 방식 고질적 문제 부각\n거래규모는 대부...,https://n.news.naver.com/mnews/article/009/000...,,매일경제,김정석 기자,경제,2026-02-06 09:28:15
2,ART_20260206195515,"""원화약세 막자""… 정부, 외평채 17년만에 최대규모 30억弗 발행","당국, 원화값 방어 총력전\n외국인 증시서 역대급 순매도\n원화값 장중 1474원까...",https://n.news.naver.com/mnews/article/009/000...,,매일경제,김혜란 외 2명,경제,2026-02-06 17:38:08
3,ART_20260206195518,9일부터 스마트폰으로 로또 산다… 20여년 묶인 빗장 풀려,뉴시스.\n\n[파이낸셜뉴스] 오는 9일부터 스마트폰으로 로또복권을 구매할 수 있게...,https://n.news.naver.com/mnews/article/014/000...,,파이낸셜뉴스,김찬미 기자,경제,2026-02-06 12:00:24
4,ART_20260206195520,"코스피 낙폭 줄여 5,000선 회복...한때 매도 사이드카",코스피가 미국 증시 약세 여파에 5% 넘게 급락했다가 5천 선을 회복했습니다.\n\...,https://n.news.naver.com/mnews/article/052/000...,,YTN,손효정 기자,경제,2026-02-06 14:55:08
5,ART_20260206195522,반도체·서학개미의 힘… 작년 경상흑자 1230억달러 사상최대,■ 전년대비 23% 급증\n\n상품수지 24% 늘어 1380억달러\nIT업 제품 중...,https://n.news.naver.com/mnews/article/021/000...,,문화일보,박세영 기자,경제,2026-02-06 11:49:14
6,ART_20260206195524,"당기순익 5.8조 KB, '번 만큼 돌려준다'…역대 최대 규모 주주환원",'정부의 생산적 금융 조력자' 역할 충실\n비이자 비즈니스 중심의 성장\n분기별 순...,https://n.news.naver.com/mnews/article/277/000...,,아시아경제,김민영 기자,경제,2026-02-05 15:54:55
7,ART_20260206195527,쿠팡 변수에 '새벽 배송' 빗장 풀리나?...여당 내 이견도,"쿠팡, '새벽 배송' 시스템 속 유통 공룡으로 성장\n대형마트, 심야 영업 규제…전...",https://n.news.naver.com/mnews/article/052/000...,,YTN,윤웅성 기자,경제,2026-02-06 18:29:29
8,ART_20260206195529,"현대차그룹, 미 3대 유력 매체 어워즈 석권...""팰리세이드 2관왕""",현대차 팰리세이드 〈사진 제공=현대차그룹〉\n현대차 SUV 팰리세이드가 미국 2개 ...,https://n.news.naver.com/mnews/article/437/000...,,JTBC,박소연 기자,경제,2026-02-06 11:22:17
9,ART_20260206195531,'심리적 저항선' 무너진 비트코인…6만6000달러까지 추락,Getty Images Bank\n\n가상화폐 시가총액 1위 비트코인 가격이 연일 ...,https://n.news.naver.com/mnews/article/015/000...,,한국경제,이송렬 기자,경제,2026-02-06 06:55:10


In [9]:
output_filename = f"saved/Articles_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
df_articles.to_excel(output_filename, index=False, engine='openpyxl', )

In [10]:
driver.quit()